In [1]:
from google.colab import drive
drive.mount('/content/drive/') 

Mounted at /content/drive/


In [1]:
import pandas as pd

environmental_data_atlas = pd.read_csv('/content/drive/Shared drives/Datathon/DATA_ATLAS.csv')
stock_company_profile = pd.read_csv('/content/drive/Shared drives/Datathon/STOCK_COMPANY_PROFILE.csv')
stock_history = pd.read_csv('/content/drive/Shared drives/Datathon/STOCK_HISTORY.csv')
stock_symbols = pd.read_csv('/content/drive/Shared drives/Datathon/STOCK_SYMBOLS.csv')

In [2]:
def year_avg_close_price(company, year):
  df = stock_history[stock_history['SYMBOL'] == company]
  df = df[df['DATE'].str.contains(str(year))]
  return df['ADJCLOSE'].mean()

In [3]:
def year_avg_volume(company, year):
  df = stock_history[stock_history['SYMBOL'] == company]
  df = df[df['DATE'].str.contains(str(year))]
  return df['VOLUME'].mean()

In [4]:
def average_environmental_data(year):
  indicators = environmental_data_atlas['Indicator'].unique().tolist()
  val_list = []
  for indicator in indicators:
    df = environmental_data_atlas[environmental_data_atlas['Year'] == year]
    val = df[environmental_data_atlas['Indicator'] == indicator]['Value'].sum()
    val_list.append(val)
  return val_list

In [ ]:
x_vector = []
df = stock_history[stock_history['SYMBOL'] == 'AAPL']
earliest_stock = df['DATE'].min()
earliest_stock = earliest_stock[:4]
latest_stock = df['DATE'].max()
latest_stock = latest_stock[:4] if int(latest_stock[:4]) <= 2019 else 2019
for i in range(int(earliest_stock), int(latest_stock) + 1):
  temp = [i, year_avg_close_price('AAPL', i), year_avg_volume('AAPL', i)] + average_environmental_data(i)
  x_vector.append(temp)

# labels
y_vector = []
for i in range(int(earliest_stock), int(latest_stock) + 1):
  y_vector.append(year_avg_close_price('AAPL', i))

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_vector, y_vector, shuffle=False)

In [21]:
# add columns to x_train and x_test
column_names = [' ', 'AVGYearlyClose']
for i in range(len(x_vector[0]) - 2):
  column_names.append(' ')

xtest_df = pd.DataFrame(x_test, columns=column_names)
xtrain_df = pd.DataFrame(x_train, columns=column_names)

In [22]:
xtest_df.to_csv("/content/drive/Shared drives/Datathon/autogluon_x_test.csv")
xtrain_df.to_csv("/content/drive/Shared drives/Datathon/autogluon_x_train.csv")

In [23]:
test = pd.read_csv('/content/drive/Shared drives/Datathon/autogluon_x_train.csv')
test.head()

,Unnamed: 0,,AVGYearlyClose,.1,.2,.3,.4,.5,.6,.7,.8,.9,.10,.11,.12,.13,.14,.15,.16,.17,.18,.19,.20,.21,.22,.23,.24,.25,.26,.27,.28,.29,.30,.31,.32,.33,.34,.35,.36,.37,.38,.39,.40,.41,.42,.43,.44,.45,.46,.47
0,0,1980,0.107222,1.034501e+08,18091.80037,1208.74617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4120.71176,254.81663,3181.09850,2723.16759,1285.18072,3218.61769,589.60718,7551732.155,1977.64737,9547300.083,11820.10797,2554164.509,977.87869,3.071149e+06,9966.76698,1.568752e+06,12254.43371,5.991702e+06,1.778504e+06,2.704464e+06,190644.57852,4.925495e+06,0.0,3583001.0,491804.0,84346.0,79733.0,104245.0,0.0,0.0,0.0
1,1,1981,0.085626,3.252756e+07,17787.48137,1134.41685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4152.68251,243.63632,3255.65835,2715.75154,1277.78739,3271.11069,578.42577,7498272.234,1976.44079,9132716.338,11859.32974,2599209.937,1052.26974,3.026422e+06,9960.17790,1.521332e+06,12311.12862,5.851793e+06,1.727046e+06,2.608747e+06,190406.15223,4.472236e+06,0.0,3496746.0,491038.0,85016.0,58246.0,95737.0,0.0,0.0,0.0
2,2,1982,0.067423,8.444667e+07,17615.40333,1105.34378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4096.73010,237.74315,3329.35908,2647.19429,1261.85695,3293.67046,568.63327,7670003.584,2058.19139,8953865.159,11795.81588,2548576.001,1045.57262,3.069972e+06,9915.27366,1.597245e+06,12316.97682,6.269163e+06,1.762952e+06,2.778074e+06,192603.16155,6.095739e+06,0.0,3470982.0,465682.0,86407.0,58115.0,85814.0,0.0,0.0,0.0
3,3,1983,0.131972,1.759408e+08,17777.75733,1080.44134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4140.55462,238.43865,3398.21203,2603.60537,1217.94960,3286.09157,593.46652,7926593.963,2135.02805,8906376.642,11715.98979,2587578.213,1099.51030,3.061073e+06,9794.18920,1.612970e+06,12256.46280,6.120151e+06,1.793234e+06,2.753106e+06,195212.48911,5.635440e+06,0.0,3516130.0,460276.0,89872.0,52269.0,83353.0,0.0,0.0,0.0
4,4,1984,0.094400,1.659250e+08,18388.17896,1117.83456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4252.27703,233.25241,3439.16864,2579.15563,1248.28421,3267.00953,563.29662,8283522.397,2230.03724,8953451.622,11557.53199,2801881.360,1191.28291,3.033687e+06,9790.46621,1.601254e+06,12318.89175,5.966796e+06,1.829739e+06,2.667446e+06,199030.93124,4.237173e+06,0.0,3649042.0,497785.0,93603.0,45179.0,83882.0,0.0,0.0,0.0


In [24]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='AVGYearlyClose', problem_type='regression').fit(train_data='/content/drive/Shared drives/Datathon/autogluon_x_train.csv')
predictions = predictor.predict('/content/drive/Shared drives/Datathon/autogluon_x_test.csv')

No path specified. Models will be saved in: "AutogluonModels/ag-20211017_041629/"
Loaded data from: /content/drive/Shared drives/Datathon/autogluon_x_train.csv | Columns = 50 / 50 | Rows = 30 -> 30
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20211017_041629/"
AutoGluon Version:  0.3.1
Train Data Rows:    30
Train Data Columns: 49
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10972.02 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Useless Original 

[1000]	train_set's rmse: 0.0181395	valid_set's rmse: 0.763973
[2000]	train_set's rmse: 0.000699826	valid_set's rmse: 0.758573
[3000]	train_set's rmse: 3.07312e-05	valid_set's rmse: 0.758282
[4000]	train_set's rmse: 2.02959e-06	valid_set's rmse: 0.758263
[5000]	train_set's rmse: 1.48576e-07	valid_set's rmse: 0.758262
[6000]	train_set's rmse: 1.38725e-08	valid_set's rmse: 0.758262
[7000]	train_set's rmse: 1.26527e-09	valid_set's rmse: 0.758262
[8000]	train_set's rmse: 1.15965e-10	valid_set's rmse: 0.758262
[9000]	train_set's rmse: 1.08724e-11	valid_set's rmse: 0.758262
[10000]	train_set's rmse: 1.03314e-12	valid_set's rmse: 0.758262


	-0.7583	 = Validation score   (root_mean_squared_error)
	2.22s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.2124	 = Validation score   (root_mean_squared_error)
	0.31s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 8.96s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20211017_041629/")
Loaded data from: /content/drive/Shared drives/Datathon/autogluon_x_test.csv | Columns = 50 / 50 | Rows = 10 -> 10


In [29]:
from autogluon.tabular import TabularDataset
print(predictor.leaderboard(TabularDataset('/content/drive/Shared drives/Datathon/autogluon_x_test.csv')))

Loaded data from: /content/drive/Shared drives/Datathon/autogluon_x_test.csv | Columns = 50 / 50 | Rows = 10 -> 10


                  model  score_test  score_val  pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         ExtraTreesMSE  -27.128242  -0.501656        0.109473       0.102987  0.548641                 0.109473                0.102987           0.548641            1       True          7
1       RandomForestMSE  -27.255699  -0.503076        0.109509       0.103064  0.544385                 0.109509                0.103064           0.544385            1       True          5
2        NeuralNetMXNet  -27.767720  -0.445544        0.166478       0.153837  1.645616                 0.166478                0.153837           1.645616            1       True         10
3              CatBoost  -28.204631  -0.353104        0.003428       0.004560  0.439121                 0.003428                0.004560           0.439121            1       True          6
4         LightGBMLarge  -28.282731  -0.75826

In [33]:
predictor.evaluate('/content/drive/Shared drives/Datathon/autogluon_x_test.csv')

Loaded data from: /content/drive/Shared drives/Datathon/autogluon_x_test.csv | Columns = 50 / 50 | Rows = 10 -> 10
Evaluation: root_mean_squared_error on test data: -28.545271378576018
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -28.545271378576018,
    "mean_squared_error": -814.8325180765512,
    "mean_absolute_error": -24.662623471505082,
    "r2": -3.2774440072294606,
    "pearsonr": -0.8825873370730156,
    "median_absolute_error": -21.759444972967504
}


{'mean_absolute_error': -24.662623471505082,
 'mean_squared_error': -814.8325180765512,
 'median_absolute_error': -21.759444972967504,
 'pearsonr': -0.8825873370730156,
 'r2': -3.2774440072294606,
 'root_mean_squared_error': -28.545271378576018}

In [34]:
predictor.predict('/content/drive/Shared drives/Datathon/autogluon_x_test.csv')

Loaded data from: /content/drive/Shared drives/Datathon/autogluon_x_test.csv | Columns = 50 / 50 | Rows = 10 -> 10


0    2.160420
1    1.834839
2    1.901266
3    1.678495
4    1.371329
5    0.808950
6    0.556563
7    0.550500
8    0.551944
9    0.387016
Name: AVGYearlyClose, dtype: float32

In [31]:
for row in x_test:
  print(row)

[2010, 8.01928755952381, 599305266.6666666, 31540.789949999995, 1197.55244, 50.73382000000001, 1051955.22124, 45518.90000000001, 3881.17444, 6816.04091, 2062.7582199999997, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 281.6120000000001, 5329.600460000001, 2132.82244, 1290.19229, 4454.30649, 592.17222, 17322148.694, 2745.95467, 12500051.861, 12103.90253, 6237963.036, 3265.41101, 3367470.0, 0.0, 2140800.0, 0.0, 7704390.0, 2993520.0, 2741480.0, 267820.0, -750161.84094, 5448.84511, 8487571.0, 1638361.0, 438636.0, 66948.0, 286707.0, 4628.5, 933.1000000000003, 40999.09]
[2011, 11.233943111111122, 492298966.6666667, 32623.58808, 1204.93053, 51.53446, 1030683.5477100001, 42631.40000000001, 1996.8036700000002, 11002.99, 1984.4244800000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 283.72898, 5393.33426, 2161.44352, 1223.49419, 4450.438270000001, 570.2132899999999, 18482760.233, 2861.60107, 12462482.511, 11803.647130000001, 6367642.823999999, 3275.2539699999998, 3421190.0, 0.0, 2221570.0, 0.0, 7807040.